In [17]:
#!pip install pycountry
#!pip install torch
#!pip install --upgrade pip
!pip install --upgrade transformers
#!pip install fsspec==2023.5.0

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pycountry

from transformers import AutoTokenizer
from transformers import pipeline

ImportError: cannot import name 'pipeline' from 'transformers' (C:\Users\sd36194\AppData\Local\anaconda3\Lib\site-packages\transformers\__init__.py)

Dataset is from Kaggle: https://www.kaggle.com/datasets/dgoenrique/amazon-prime-movies-and-tv-shows

In [ ]:
amazon_df = pd.read_csv("titles.csv")

In [ ]:
amazon_df.head()

In [ ]:
amazon_df.shape

In [ ]:
amazon_df.isnull().sum()

In [ ]:
# get null percentage of each column
null_rate_dict = {}
for i in amazon_df.columns:
    null_rate = amazon_df[i].isna().sum() / len(amazon_df) * 100
    null_rate_dict[i] = null_rate

In [ ]:
# sort the null percentage from highest to lowest
sorted_null_rate = sorted(null_rate_dict.items(), key=lambda x:x[1], reverse = True)
print(sorted_null_rate)

In [ ]:
# only print out columns with null percentage
for i in sorted_null_rate:
    if i[1] > 0:
        print(f"{i[0]} null percentage: {round(i[1], 2)}%")

In [ ]:
amazon_df.info()

In [ ]:
round(amazon_df.describe(), 2)

In [ ]:
# all the null seasons are movie type
amazon_df[amazon_df['seasons'].isna()]['type'].unique()

In [ ]:
# fill the null values of seasons to zero
amazon_df['seasons'] = amazon_df['seasons'].fillna('0')

In [ ]:
# get the first genre from genres as our primary genre
amazon_df['primary_genre'] = amazon_df['genres'].apply(lambda x: x.strip("[ ").strip("]").split(",")[0])
amazon_df['primary_genre'] = amazon_df['primary_genre'].str.replace("'", "")
amazon_df.head()

In [ ]:
amazon_df['production_countries'].unique()

In [ ]:
# get the primary country code from the list as production country
amazon_df['production_country'] = amazon_df['production_countries'].apply(lambda x: x.strip("[ ").strip("]").split(",")[0])
amazon_df['production_country'] = amazon_df['production_country'].str.replace("'", "")
amazon_df.head()

In [ ]:
# print unique values of country code from the production country list
abbreviated_countries = amazon_df['production_country']
abbreviated_countries.unique()

In [ ]:
full_countries = []
# for each production country code, get the full country name and store in a list
for abbreviation in abbreviated_countries:
    try:
        country = pycountry.countries.get(alpha_2 = abbreviation).name
        full_countries.append(country)
    except:
        full_countries.append(abbreviation)

In [ ]:
len(full_countries)

In [ ]:
# create a new column with the full country name
amazon_df['full_country_name'] = full_countries
amazon_df

# Sentiment Analysis on Description